# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [2]:
import pandas as pd
import numpy as np
import requests
import warnings

warnings.filterwarnings('ignore')

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
#sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


In [6]:
# Verificando a quantidade de registros duplicados
novo_sinasc = sinasc.drop_duplicates()
novo_sinasc.shape
#como o valor do novo_sinasc é o mesmo do primeiro df declarado, então não há dados duplicados

(27028, 69)

In [ ]:
# 2) seu código aqui
# Verificando a quantidade de registro nulo
sinasc.isnull().sum().sort_values(ascending=False)
# Existem vários valores nulos na tabela

In [7]:
# 3) seu código aqui
df = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
df

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [18]:
# 4) seu código aqui
# verificando a quantidade de valores nulos na coluna 'APGAR5'
df['APGAR5'].isnull().sum()
# verificando a quantidade de valores nulos na coluna 'APGAR5'
df_missing = df[df['APGAR5'].isnull()]
df_sem_missing = df[~df['APGAR5'].isnull()]

# A variavel df_missing indica todas as linhas com a coluna APGAR5 nulas
# A df_sem_missing indica todas as linhas com a coluna APGAR5 preenchidas

df_missing.shape
df_sem_missing.shape

(103, 9)

In [59]:
# 5) seu código aqui
# verificando os tipos de dados
df_sem_missing['ESTCIVMAE'].unique()
# verificando a quantidade de linhas nulas
df_sem_missing['ESTCIVMAE'].isnull().sum()
# transformando elas no codigo 9
df_sem_missing['ESTCIVMAE'].fillna(9, inplace=True)
# verificando a não existencia de linhas nulas agora
df_sem_missing['ESTCIVMAE'].unique()
df_sem_missing['ESTCIVMAE'].isnull().sum()

0

In [60]:
# 6) Seu código aqui
# Transformando a quantidade de filhos = 0
df_sem_missing['QTDFILVIVO'].fillna(0, inplace=True)

In [57]:
# Conferindo se todos os valores nulos foram transformados!
df_sem_missing['QTDFILVIVO'].isnull().sum()

0

In [76]:
# 7) seu código aqui
# Verificando quais tabelas possuem valores nulos para tratar
df_sem_missing.isnull().sum()
# mostrando as colunas que possuvem valores nulos
df_sem_missing[['ESCMAE', 'GESTACAO', 'GRAVIDEZ']]
# verificando a quantidade de registro por coluna
df_sem_missing['GRAVIDEZ'].value_counts()
# como a quantidade de gravidez unica é completamente superior, defini as nulas também como unica
df_sem_missing['GRAVIDEZ'].fillna('Única', inplace=True)
# contando novamente para verificar se todos foram alterados
df_sem_missing['GRAVIDEZ'].isnull().sum()
# verificando outra coluna
df_sem_missing['GESTACAO'].value_counts()
# Novamente o valor de 37 a 41 semanas é bem superior aos demais, então também coloquei todos os nulos como 37 a 41 semanas
df_sem_missing['GESTACAO'].fillna('37 a 41 semanas', inplace=True)
# Verificando os valores
df_sem_missing['GESTACAO'].isnull().sum()
# Fazendo o mesmo procedimento para a outra coluna
df_sem_missing['ESCMAE'].value_counts()
df_sem_missing['ESCMAE'].fillna('8 a 11 anos', inplace=True)
df_sem_missing['ESCMAE'].isnull().sum()

# Verificando se possui algum valor nulo no dataset
df_sem_missing.isnull().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

In [82]:
# 8) seu código aqui
# Definindo as categorias
df_sem_missing.loc[df_sem_missing['APGAR5'] >= 8, 'categoria'] = 'normal'
df_sem_missing.loc[(df_sem_missing['APGAR5'] >= 6) & (df_sem_missing['APGAR5'] <= 7), 'categoria'] = 'asfixia leve'
df_sem_missing.loc[(df_sem_missing['APGAR5'] >= 4) & (df_sem_missing['APGAR5'] <= 6), 'categoria'] = 'asfixia moderada'
df_sem_missing.loc[df_sem_missing['APGAR5'] < 4, 'categoria'] = 'asfixia severa'
# Conferindo os valores!
df_sem_missing['categoria'].value_counts()

normal              26470
asfixia leve          236
asfixia moderada      146
asfixia severa         73
Name: categoria, dtype: int64

In [4]:
# 9) seu código aqui
# Criando um novo dataframe para renomear as colunas
dataframe = pd.read_csv('SINASC_RO_2019.csv')
# Renomeando as colunas
dataframe.rename(columns=lambda x: x.lower().replace(" ", "_"), inplace=True)
# Verificando os valores das colunas
print(dataframe.columns)


Index(['origem', 'codestab', 'codmunnasc', 'locnasc', 'idademae', 'estcivmae',
       'escmae', 'codocupmae', 'qtdfilvivo', 'qtdfilmort', 'codmunres',
       'gestacao', 'gravidez', 'parto', 'consultas', 'dtnasc', 'horanasc',
       'sexo', 'apgar1', 'apgar5', 'racacor', 'peso', 'idanomal', 'dtcadastro',
       'codanomal', 'numerolote', 'versaosist', 'dtrecebim', 'difdata',
       'dtrecoriga', 'naturalmae', 'codmunnatu', 'codufnatu', 'escmae2010',
       'seriescmae', 'dtnascmae', 'racacormae', 'qtdgestant', 'qtdpartnor',
       'qtdpartces', 'idadepai', 'dtultmenst', 'semagestac', 'tpmetestim',
       'consprenat', 'mesprenat', 'tpapresent', 'sttrabpart', 'stcesparto',
       'tpnascassi', 'tpfuncresp', 'tpdocresp', 'dtdeclarac', 'escmaeagr1',
       'stdnepidem', 'stdnnova', 'codpaisres', 'tprobson', 'paridade',
       'kotelchuck', 'contador', 'munresstatus', 'munrestipo', 'munresnome',
       'munresuf', 'munreslat', 'munreslon', 'munresalt', 'munresarea'],
      dtype='object')
